In [1]:
#Run this cell if any import errors occur
#! pip install datasets transformers
#! pip install -U scikit-learn

In [2]:
#Collecting all imports
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForMultipleChoice, TrainingArguments, Trainer
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch
from sklearn.metrics import f1_score
import numpy as np

import transformers

print(transformers.__version__)
#Transformers should be at least 4.11.0 required!

4.36.2


We must now load the dataset from our local JSON files. 

In [3]:
#Load the dataset as train, validation, and test.
#We use the dev data as validation.

datasets = load_dataset("json", data_files={'train':'QQA Data/QQA_train.json', 
                                           'validation':'QQA Data/QQA_dev.json', 
                                           'test':'QQA Data/QQA_test.json'})

#Printing the dataset contents
print('Q: ' + datasets['train'][0]['question'])
print('O1: ' + datasets['train'][0]['Option1'])
print('O2: ' + datasets['train'][0]['Option2'])
print('A: ' + datasets['train'][0]['answer'])

Q: The ranger and the rustler both were riding horses that galloped at the same speed.  The rustler left at 01:00 where as the ranger left at 0500 hours. Who has traveled further?? 
O1: the ranger
O2: the rustler
A: Option 2


In [4]:
#Print dataset structure
datasets

DatasetDict({
    train: Dataset({
        features: ['question_sci_10E', 'question', 'Option2', 'question_char', 'answer', 'Option1', 'question_mask', 'type', 'question_sci_10E_char'],
        num_rows: 564
    })
    validation: Dataset({
        features: ['question_sci_10E', 'question', 'Option2', 'question_char', 'answer', 'Option1', 'question_mask', 'type', 'question_sci_10E_char'],
        num_rows: 81
    })
    test: Dataset({
        features: ['question_sci_10E', 'question', 'Option2', 'question_char', 'answer', 'Option1', 'question_mask', 'type', 'question_sci_10E_char'],
        num_rows: 162
    })
})

The imported data is of the form:

{"question": "Jame's mother has a photo of Jane standing at a height of 14 inches, whereas a mountain appears to have height of 26 cm. It looks that way because? ", "Option1": "the mountain was farther away", "Option2": "Jane was farther away", "answer": "Option 2", "type": "Type_3", "question_sci_10E": "Jame's mother has a photo of Jane standing at a height of 1.4000000000E+01 inches, whereas a mountain appears to have height of 2.6000000000E+01 cm. It looks that way because? ", "question_char": "Jame's mother has a photo of Jane standing at a height of 1 4 inches, whereas a mountain appears to have height of 2 6 cm. It looks that way because? ", "question_sci_10E_char": "Jame's mother has a photo of Jane standing at a height of 1 . 4 0 0 0 0 0 0 0 0 0 E + 0 1 inches, whereas a mountain appears to have height of 2 . 6 0 0 0 0 0 0 0 0 0 E + 0 1 cm. It looks that way because? ", "question_mask": "Jame's mother has a photo of Jane standing at a height of [Num] inches, whereas a mountain appears to have height of [Num] cm. It looks that way because? "

In [5]:
#Let's get rid of the other columns
datasets = datasets.remove_columns(['question_char', 'question_sci_10E',
                         'question_sci_10E_char',
                         'question_mask', 'type',])

#We now only have a question, answer, and 2 options.

In [6]:
#Renaming for compatibility later
datasets = datasets.rename_column('answer', 'label')

For evaluation, see https://huggingface.co/spaces/evaluate-metric/accuracy

For fine-tuning, see: https://huggingface.co/docs/transformers/en/training#train-with-pytorch-trainer

For handing multiple choice, see https://huggingface.co/docs/transformers/en/tasks/multiple_choice


In [7]:
#Set correct answer to an integer. 

def set_labels(example):
    #print(example)
    example["label"] = int(example["label"][-1]) - 1
    return example

datasets = datasets.map(set_labels)

In [8]:
datasets["train"][30]

{'question': 'A race car and a pickup both drove on the highway at the same speed. The driver of the race car got tired and parked after 29 mins, while the driver of the pickup ran for 43 mins. Which vehicle ultimately went the greater distance?? ',
 'Option2': 'pickup',
 'label': 1,
 'Option1': 'race car'}

In [9]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', use_fast=True)
tokenizer("This is a boat.", "This is a plane.")

{'input_ids': [101, 2023, 2003, 1037, 4049, 1012, 102, 2023, 2003, 1037, 4946, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [10]:
ending_names = ["Option1", "Option2"]

def preprocess_function(examples):
    # Repeat each first sentence two times to go with the two possibilities of second sentences.
    first_sentences = [[context] * 2 for context in examples["question"]]
    # Grab all second sentences possible for each context.
    question_headers = examples["question"]
    second_sentences = [[f"{header} {examples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)]
    
    # Flatten everything
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    # Tokenize
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    # Un-flatten
    return {k: [v[i:i+2] for i in range(0, len(v), 2)] for k, v in tokenized_examples.items()}

In [11]:
examples = datasets["train"][:5]
features = preprocess_function(examples)
print(len(features["input_ids"]), len(features["input_ids"][0]), [len(x) for x in features["input_ids"][0]])

5 2 [89, 90]


In [12]:
idx = 3
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(2)]

['[CLS] a tank weighs around 63 tons. a toy car weighs 1. 5 kg. because of this? [SEP] a tank weighs around 63 tons. a toy car weighs 1. 5 kg. because of this? the tank will speed up faster than the toy car [SEP]',
 '[CLS] a tank weighs around 63 tons. a toy car weighs 1. 5 kg. because of this? [SEP] a tank weighs around 63 tons. a toy car weighs 1. 5 kg. because of this? the toy car will speed up faster than the tank [SEP]']

In [13]:
#encoded_datasets = datasets.map(preprocess_function, batched=True)
#print(encoded_datasets)

In [14]:
model = AutoModelForMultipleChoice.from_pretrained('bert-base-uncased')

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        
        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [16]:
#Testing stuff. Be sure to create encoded_datasets if you're running this. 
#accepted_keys = ["input_ids", "attention_mask", "label"]
#features = [{k: v for k, v in encoded_datasets["train"][i].items() if k in accepted_keys} for i in range(10)]
#batch = DataCollatorForMultipleChoice(tokenizer)(features)
#[tokenizer.decode(batch["input_ids"][8][i].tolist()) for i in range(2)]

In [17]:
#Use the F1 score metric to evaluate our predictions. 

#Old evaluator.
'''
def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}
'''

#New evaluator.
def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    
    preds = np.argmax(predictions, axis=1)

    print(preds)
    return {"accuracy": f1_score(preds, label_ids, average='micro').astype(np.float32).mean().item()}

In [18]:
'''
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer),
    compute_metrics=compute_metrics,
)
trainer.train()
'''

'\ntrainer = Trainer(\n    model,\n    args,\n    train_dataset=encoded_datasets["train"],\n    eval_dataset=encoded_datasets["validation"],\n    tokenizer=tokenizer,\n    data_collator=DataCollatorForMultipleChoice(tokenizer),\n    compute_metrics=compute_metrics,\n)\ntrainer.train()\n'

In [19]:
# A cell to reset the dataset if/when problems occur with it.
# Probably good to run this if you ran the above cells, since there's a lot of testing stuff for one model

datasets = load_dataset("json", data_files={'train':'QQA Data/QQA_train.json', 
                                           'validation':'QQA Data/QQA_dev.json', 
                                           'test':'QQA Data/QQA_test.json'})

#Printing the dataset contents
print('Q: ' + datasets['train'][0]['question'])
print('O1: ' + datasets['train'][0]['Option1'])
print('O2: ' + datasets['train'][0]['Option2'])
print('A: ' + datasets['train'][0]['answer'])

#Let's get rid of the other columns
datasets = datasets.remove_columns(['question_char', 'question_sci_10E',
                         'question_sci_10E_char',
                         'question_mask', 'type',])

#We now only have a question, answer, and 2 options.

datasets = datasets.rename_column('answer', 'label')

def set_labels(example):
    #print(example)
    example["label"] = int(example["label"][-1]) - 1
    return example

datasets = datasets.map(set_labels)

print(datasets)

Q: The ranger and the rustler both were riding horses that galloped at the same speed.  The rustler left at 01:00 where as the ranger left at 0500 hours. Who has traveled further?? 
O1: the ranger
O2: the rustler
A: Option 2
DatasetDict({
    train: Dataset({
        features: ['question', 'Option2', 'label', 'Option1'],
        num_rows: 564
    })
    validation: Dataset({
        features: ['question', 'Option2', 'label', 'Option1'],
        num_rows: 81
    })
    test: Dataset({
        features: ['question', 'Option2', 'label', 'Option1'],
        num_rows: 162
    })
})


In [20]:
#Training function. Duplicate and change model name for other models.

modelName = ""

def autoTrain(model_name = 'bert-base-uncased', batch_size = 16):
    global model
    global tokenizer
    global modelName
    modelName = model_name
    model = AutoModelForMultipleChoice.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    
    encoded_datasets = datasets.map(preprocess_function, batched=True)
    
    args = TrainingArguments(
        f"{model_name}-finetuned-QQA",
        evaluation_strategy = "epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=3,
        weight_decay=0.01,
    )
    
    trainer = Trainer(
        model,
        args,
        train_dataset=encoded_datasets["train"],
        eval_dataset=encoded_datasets["validation"],
        tokenizer=tokenizer,
        data_collator=DataCollatorForMultipleChoice(tokenizer),
        compute_metrics=compute_metrics,
    )
    
    trainer.train()


In [31]:

autoTrain("bert-base-uncased", 16)

#autoTrain('deepset/roberta-base-squad2', 128)

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [21]:
#Cell for testing
print(datasets["test"])

Dataset({
    features: ['question', 'Option2', 'label', 'Option1'],
    num_rows: 162
})


In [22]:
#Will tune this to have it evaluate. 

prompt = "I have 5 bagels and Joe has 2. Who has more bagels?"
candidate1 = "Me"
candidate2 = "Joe"


inputs = tokenizer([[prompt, candidate1], [prompt, candidate2]], return_tensors="pt", padding=True)
labels = torch.tensor(0).unsqueeze(0)
outputs = model(**{k: v.unsqueeze(0) for k, v in inputs.items()}, labels=labels)
logits = outputs.logits

predicted_class = logits.argmax().item()
predicted_class

#Note that it will output a 0 or 1, where 0 = Option 1 and 1 = Option 2

0

In [28]:
#Model the most recently trained model
def evaluate_hf_model():

    global model
    global tokenizer

            
    predictions = []
    references = []

    # Get predictions, and save corresponding reference (if we were using the whole dataset, we wouldn't need this step)
    for ex in datasets["test"]:

        #Based on the above cell, get 
        prompt = ex['question']
        candidate1 = ex['Option1']
        candidate2 = ex['Option2']
        
        inputs = tokenizer([[prompt, candidate1], [prompt, candidate2]], return_tensors="pt", padding=True)
        labels = torch.tensor(0).unsqueeze(0)
        outputs = model(**{k: v.unsqueeze(0) for k, v in inputs.items()}, labels=labels)
        logits = outputs.logits
        
        predicted_class = logits.argmax().item()
        predicted_class


        predictions.append(predicted_class)
        references.append(ex['label'])

    # Compute metrics
    global modelName
    print('Performance of {} : {}'.format(modelName, f1_score(predictions, references, average='micro')))

evaluate_hf_model()

Performance of amogus : 0.4567901234567901


In [ ]:
#Storing the name and accuracy to avoid having to re-evaluate everything all the time.
#Also good for graphing. 

modelNameAcc = {
    ["name", "evaluation accuracy"]
}

In [ ]:
#Plot model names, accuracies, on a graph for easy comparison